In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Carga de datos y filtrado

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")

# Elimina todas las variables meteorológicas que no correspondan a la localización de Sotavento (la localización 13)
wind_ava = wind_ava.filter(regex='^(datetime|energy|.*\.13)$')

# Elimina las variables 'lai_hv.13' y 'v10n.13'
if 'lai_hv.13' in wind_ava.columns and 'v10n.13' in wind_ava.columns:
    wind_ava = wind_ava.drop(['lai_hv.13', 'v10n.13'], axis=1)

# Vuelve a definir las variables numéricas
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns

# Quitamos "energy" de las variables numéricas, ya que querremos usarla como variable objetivo
numerical_vars = numerical_vars.drop('energy')

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(wind_ava[numerical_vars], wind_ava['energy'], test_size=1/3, random_state=100472166, shuffle=False)

# Crea el escalador
scaler = StandardScaler()

# Ajusta el escalador a las características del conjunto de entrenamiento y transformarlas
X_train_scaled = scaler.fit_transform(X_train)

# Transforma las características del conjunto de prueba usando el mismo escalador
X_test_scaled = scaler.transform(X_test)

# Divide los datos de entrenamiento en un conjunto de entrenamiento más pequeño y un conjunto de validación
X_train_inner, X_val, y_train_inner, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=100472166)

# Define el modelo SVM con los hiperparámetros seleccionados
best_model = SVR(C=1000.0, coef0= 0, gamma= 'scale', kernel= 'rbf', shrinking= True)

# Ajusta el modelo SVM en el conjunto de entrenamiento más pequeño
best_model.fit(X_train_inner, y_train_inner)

# Apartado 6

En primer lugar utilizaremos nuestro mejor modelo hasta el momento para comprobar si las predicciones para valores altos son peores que para valores bajos (o viceversa). Esto lo haremos comparando su RSME

In [ ]:
from sklearn.metrics import mean_squared_error

# Predice los valores de la variable objetivo para el conjunto de validación
y_pred = best_model.predict(X_val)

# Calcula el error cuadrático medio (RMSE)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE: {rmse}")

# Divide las predicciones y los valores reales en grupos de altos y bajos
# Aquí, asumimos que "altos" son aquellos valores mayores a la mediana y "bajos" son los menores
median_energy = np.median(y_val)
y_val_high = y_val[y_val > median_energy]
y_val_low = y_val[y_val <= median_energy]
y_pred_high = y_pred[y_val > median_energy]
y_pred_low = y_pred[y_val <= median_energy]

# Calcula el RMSE para los valores altos y bajos
rmse_high = np.sqrt(mean_squared_error(y_val_high, y_pred_high))
rmse_low = np.sqrt(mean_squared_error(y_val_low, y_pred_low))

print(f"RMSE para valores altos: {rmse_high}")
print(f"RMSE para valores bajos: {rmse_low}")

# Comprueba si las predicciones para valores altos son peores que para valores bajos
if rmse_high > rmse_low:
    print("Las predicciones para valores altos son peores que para valores bajos.")
else:
    print("Las predicciones para valores altos no son necesariamente peores que para valores bajos.")

# Crea una figura con un tamaño específico (ancho, alto)
fig, ax = plt.subplots(figsize=(6, 3))

# Crear una lista con los nombres de los grupos
groups = ['RMSE General', 'RMSE Valores Altos', 'RMSE Valores Bajos']

# Crear una lista con los RMSE de cada grupo
rmse_values = [rmse, rmse_high, rmse_low]

# Crear un diagrama de barras para los RMSE
ax.bar(groups, rmse_values, color=['blue', 'green', 'red'])
ax.set_title('Comparación de RMSE')
ax.set_ylabel('RMSE')

# Mostrar la figura
plt.tight_layout()
plt.show()

## Conversión a clasificación

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Eliminar la columna "datetime" del conjunto de datos
wind_ava.drop(["datetime"], axis=1, inplace=True)

# Determinar el tercer cuantil de la variable objetivo
energy_quantile = wind_ava['energy'].quantile(0.25)

# Clasificar los valores de energía como "baja" o "alta"
wind_ava['energy_class'] = ['baja' if e < energy_quantile else 'alta' for e in wind_ava['energy']]

# Convertir las etiquetas de clase a números
label_encoder = LabelEncoder()
wind_ava['energy_class_encoded'] = label_encoder.fit_transform(wind_ava['energy_class'])

# Almacenamos en una variable todas las columnas no relacionadas con la variable objetivo
excluir_columnas = ['energy', 'energy_class', 'energy_class_encoded']
numerical_vars = [col for col in wind_ava.columns if col not in excluir_columnas]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    wind_ava[numerical_vars], 
    wind_ava['energy_class_encoded'], 
    test_size=1/3, 
    random_state=100472166
)

# Ajustar el escalador a las características del conjunto de entrenamiento y transformarlas
scaler = StandardScaler()
X_train_class_scaled = scaler.fit_transform(X_train_class)
X_test_class_scaled = scaler.transform(X_test_class)

# Definir y entrenar un modelo de clasificación
classifier = RandomForestClassifier(random_state=100472166)
classifier.fit(X_train_class_scaled, y_train_class)

# Predecir las clases para el conjunto de prueba
y_pred_class = classifier.predict(X_test_class_scaled)

# Calcular la precisión del modelo de clasificación
accuracy = classifier.score(X_test_class_scaled, y_test_class)
print(f"Precisión del modelo de clasificación: {accuracy}")

## Hiperparámetros y Classification Report

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(
    wind_ava[numerical_vars], 
    wind_ava['energy_class_encoded'], 
    test_size=1/3, 
    random_state=100472166
)

# Definir el modelo
model = RandomForestClassifier(random_state=100472166)

# Definir los hiperparámetros a ajustar
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Realizar la búsqueda en cuadrícula para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros: ", grid_search.best_params_)

# Entrenar el modelo con los mejores hiperparámetros
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Predecir las clases para el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Calcular y mostrar las métricas
print(classification_report(y_test, y_pred))


# Crear una figura con un tamaño específico (ancho, alto)
fig, ax = plt.subplots(figsize=(12, 6))

# Crear un mapa de calor para la matriz de confusión
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)

# Configurar los títulos y las etiquetas
ax.set_title('Matriz de Confusión')
ax.set_xlabel('Predicción')
ax.set_ylabel('Real')

# Mostrar la figura
plt.tight_layout()
plt.show()

## Modelos distintos

Podemos probar a realizar el ajuste de hiperparámetros también para otros modelos diferentes al Random Forest Classifier, como SVC o los árboles de decisión

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Hiperparámetros para SVC
svc_params = {
    'C': np.logspace(-3, 3, 7),
    'gamma': np.logspace(-3, 3, 7),
    'shrinking': [True, False]
    # 'kernel': ['linear', 'rbf']
}

# Hiperparámetros para DecisionTreeClassifier
dt_params = {
    'max_depth': np.arange(1, 15),
    'min_samples_split': np.arange(2, 15),
    'min_samples_leaf': np.arange(1, 15),
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
}

# Hiperparámetros para KNeighborsClassifier
knn_params = {
    'n_neighbors': np.arange(1, 15),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'leaf_size': np.arange(1, 15),
    'p': [1, 2]
}

# Ajustar el escalador a las características del conjunto de entrenamiento y transformarlas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SVC
svc = SVC(random_state=100472166)
svc_search = GridSearchCV(svc, svc_params, cv=5, scoring='accuracy', n_jobs=-1)
svc_search.fit(X_train_scaled, y_train)
y_pred_svc = svc_search.predict(X_test_scaled)
cm_svc = confusion_matrix(y_test, y_pred_svc)
print("Mejores hiperparámetros para SVC:", svc_search.best_params_)
print(classification_report(y_test, y_pred_svc))
print(cm_svc)
print("------------------------")

# DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=100472166)
dt_search = GridSearchCV(dt, dt_params, cv=5, scoring='accuracy', n_jobs=-1)
dt_search.fit(X_train_scaled, y_train)
y_pred_dt = dt_search.predict(X_test_scaled)
cm_dt = confusion_matrix(y_test, y_pred_dt)
print("Mejores hiperparámetros para DecisionTreeClassifier:", dt_search.best_params_)
print(classification_report(y_test, y_pred_dt))
print(cm_dt)
print("------------------------")

# KNeighborsClassifier
knn = KNeighborsClassifier()
knn_search = GridSearchCV(knn, knn_params, cv=5, scoring='accuracy', n_jobs=-1)
knn_search.fit(X_train_scaled, y_train)
y_pred_knn = knn_search.predict(X_test_scaled)
print("Mejores hiperparámetros para KNeighborsClassifier:", knn_search.best_params_)
print(classification_report(y_test, y_pred_knn))
cm_knn = confusion_matrix(y_test, y_pred_knn)
print(cm_knn)
print("------------------------")

# Crear una figura con un tamaño específico (ancho, alto)
fig, axs = plt.subplots(3, figsize=(12, 18))

# Crear mapas de calor para las matrices de confusión
sns.heatmap(cm_svc, annot=True, fmt='d', cmap='Blues', ax=axs[0])
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Blues', ax=axs[1])
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Blues', ax=axs[2])

# Configurar los títulos y las etiquetas
axs[0].set_title('Matriz de Confusión para SVC')
axs[0].set_xlabel('Predicción')
axs[0].set_ylabel('Real')

axs[1].set_title('Matriz de Confusión para DecisionTreeClassifier')
axs[1].set_xlabel('Predicción')
axs[1].set_ylabel('Real')

axs[2].set_title('Matriz de Confusión para KNeighborsClassifier')
axs[2].set_xlabel('Predicción')
axs[2].set_ylabel('Real')

# Mostrar la figura
plt.tight_layout()
plt.show()

# Apartado 7

Uno de los usos que se le ha dado a ChatGPT ha sido conseguir bases de las que partir. Obviamente tuvimos que reescribir prácticamente todo ese código, ya que ChatGPT generaba un código muy incompleto y que nunca exploraba opciones más allá de lo que se le pedía. Sin embargo, esto nos permitió ahorrar tiempo al no tener que escribir a mano todas las tareas sencillas y repetitivas, tiempo que pudimos usar en seguir mejorando nuestros modelos finales.

También nos fue de gran utilidad cuando lo usamos para explorar opciones más allá de las vistas en clase. Este fue el caso, por ejemplo, del ajuste de hiperparámetros. Para conseguir los mejores resultados posibles, le pedimos a ChatGPT que nos dijese todos los posibles parámetros que admitía cada modelo y cuáles eran los valores que solían tomar. Después de eso, fuimos probando con varios de ellos hasta que estuvimos contentos con los que teníamos, ya fuera en términos de MSE o de tiempo de ejecución. Por lo tanto, esto no solo nos permitió ajustar el modelo con más precisión, si no que además ahorramos tiempo al ya tener una idea de los rangos de valores y no tener que probarlos todos.

Algunos de los prompts en cuestión fueron:

## Prompt 1

¿En un modelo de regresión qué HPO admiten los modelos SVM, KNN, Árboles de decisión, Regresión lineal y Lasso? Dame todos los posibles parametros que se puedan ajustar.

*Cabe destacar que al realizar esta pregunta el ChatGPT confundía hiperparámetros y valores que podrían tomar entre los diferentes tipos de modelos, pero al final obtuvimos esta respuesta:*

    Respuesta:

SVM (Support Vector Machines)

- **C**: Parámetro de regularización.
- **kernel**: Tipo de kernel a utilizar.
- **degree**: Grado del polinomio en el caso de 'poly'.
- **gamma**: Coeficiente para el kernel 'rbf', 'poly' y 'sigmoid'.
- **coef0**: Coeficiente independiente en 'poly' y 'sigmoid'.
- **shrinking**: Especifica si se debe utilizar el algoritmo de reducción.
- **probability**: Especifica si se debe calcular la probabilidad ajustada.
- **tol**: Tolerancia para la parada del algoritmo.
- **cache_size**: Tamaño de la caché en MB.
- **class_weight**: Pesos de las clases.
- **verbose**: Nivel de verbosidad.
- **max_iter**: Número máximo de iteraciones.
- **decision_function_shape**: Forma de la función de decisión.
- **break_ties**: Método para romper empates en la decisión.
- **random_state**: Semilla para la generación de números aleatorios.

KNN (K-Nearest Neighbors)

- **n_neighbors**: Número de vecinos a considerar.
- **weights**: Función de ponderación de los vecinos.
- **algorithm**: Algoritmo utilizado para calcular los vecinos más cercanos.
- **leaf_size**: Tamaño de los árboles construidos por el algoritmo 'ball_tree' o 'kd_tree'.
- **p**: Parámetro de la métrica de distancia.
- **metric**: Métrica de distancia utilizada.
- **metric_params**: Parámetros adicionales para la métrica de distancia.
- **n_jobs**: Número de trabajos paralelos.

Árboles de Decisión (Decision Trees)

- **criterion**: Función a utilizar para medir la calidad de una división.
- **splitter**: Algoritmo a utilizar para encontrar la división de los nodos.
- **max_depth**: Profundidad máxima del árbol.
- **min_samples_split**: Número mínimo de muestras requeridas para dividir un nodo interno.
- **min_samples_leaf**: Número mínimo de muestras requeridas para estar en un nodo hoja.
- **min_weight_fraction_leaf**: Fración mínima de peso total en un nodo hoja.
- **max_features**: Número de características a considerar al buscar la mejor división.
- **random_state**: Semilla para la generación de números aleatorios.
- **max_leaf_nodes**: Número máximo de nodos hoja.
- **min_impurity_decrease**: Impureza mínima requerida para una división.
- **class_weight**: Pesos de las clases.
- **presort**: Especifica si se debe preordenar los datos.

Regresión Lineal (Linear Regression)

- **fit_intercept**: Especifica si se debe ajustar un término de intercepción.
- **normalize**: Especifica si se debe normalizar los datos antes del ajuste.
- **copy_X**: Especifica si se debe copiar los datos de entrada.
- **n_jobs**: Número de trabajos paralelos.
- **positive**: Especifica si se debe asegurar que los coeficientes sean positivos.

Lasso (L1 Regularization)

- **alpha**: Coeficiente de regularización.
- **fit_intercept**: Especifica si se debe ajustar un término de intercepción.
- **normalize**: Especifica si se debe normalizar los datos antes del ajuste.
- **precompute**: Especifica si se debe precalcular la matriz de Gram.
- **max_iter**: Número máximo de iteraciones.
- **tol**: Tolerancia para la parada del algoritmo.
- **warm_start**: Especifica si se debe reutilizar la solución anterior como punto de partida.
- **positive**: Especifica si se debe asegurar que los coeficientes sean positivos.
- **random_state**: Semilla para la generación de números aleatorios.
- **selection**: Método de selección de características.

## Prompt 2

¿Cómo puedo agrupar los gráfios que se generan en el siguiente código de forma individual en uno solo?

In [ ]:
# Comentado para que no se ejecute
'''import matplotlib.pyplot as plt

for col in numerical_vars:
    plt.figure(figsize=(10, 6))
    wind_ava[col].hist(bins=30, alpha=0.5, label=col)
    plt.title(f'Distribución de {col}')
    plt.xlabel(col)
    plt.ylabel('Frecuencia')
    plt.legend(loc='upper right')
    plt.show() '''

    Respuesta:

In [ ]:
# Comentado para que no se ejecute
'''# Número de variables numéricas
num_vars = len(numerical_vars)

# Calcular el número de filas y columnas para los subplots
num_cols = 4  # número máximo de columnas
num_rows = np.ceil(num_vars / num_cols).astype(int)

# Crear una figura y un conjunto de subtramas
fig, axs = plt.subplots(num_rows, num_cols, figsize=(6*num_cols, 4*num_rows))

# Aplanar el array de subplots para facilitar su recorrido
axs = axs.flatten()

# Para cada variable numérica, crear un histograma en su respectivo subplot
for i, col in enumerate(numerical_vars):
    axs[i].hist(wind_ava[col], bins=30, alpha=0.5, label=col)
    axs[i].set_title(f'Distribución de {col}')
    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Frecuencia')
    axs[i].legend(loc='upper right')

# Eliminar los subplots vacíos si el número de variables no es múltiplo de 4
if num_vars % num_cols != 0:
    for j in range(i+1, num_rows*num_cols):
        fig.delaxes(axs[j])

# Ajustar el espacio entre los subplots
plt.tight_layout()

# Mostrar la figura con todos los histogramas
plt.show()'''